<a href="https://colab.research.google.com/github/dmarx/advent-of-code_2021/blob/main/%5BAoC_21%5D_Day2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Day 2: Dive!

## Part 1

It seems like the submarine can take a series of commands like forward 1, down 2, or up 3:

`forward X` increases the horizontal position by X units.  
`down X` increases the depth by X units.  
`up X` decreases the depth by X units.  

Note that since you're on a submarine, down and up affect your depth, and so they have the opposite result of what you might expect.

The submarine seems to already have a planned course (your puzzle input). You should probably figure out where it's going. For example:

```
forward 5
down 5
forward 8
up 3
down 8
forward 2
```


In [1]:
test_input = """
forward 5
down 5
forward 8
up 3
down 8
forward 2
""".strip()


Your horizontal position and depth both start at 0. The steps above would then modify them as follows:

`forward 5` adds 5 to your horizontal position, a total of 5.  
`down 5` adds 5 to your depth, resulting in a value of 5.  
`forward 8` adds 8 to your horizontal position, a total of 13.  
`up 3` decreases your depth by 3, resulting in a value of 2.  
`down 8` adds 8 to your depth, resulting in a value of 10.  
`forward 2` adds 2 to your horizontal position, a total of 15.  

After following these instructions, you would have a horizontal position of 15 and a depth of 10. (Multiplying these together produces 150.)

Calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [20]:
from typing import List, Tuple

Command = Tuple[str, int]
CommandsList = List[Command]

def parse_input(in_str: str) -> CommandsList:
  recs = []
  commands = in_str.split('\n')
  for command in commands:
    direction, value = command.split(' ')
    value = int(value)
    recs.append((direction, value))
  return recs 

test_parsed = parse_input(test_input)
test_parsed

[('forward', 5),
 ('down', 5),
 ('forward', 8),
 ('up', 3),
 ('down', 8),
 ('forward', 2)]

In [22]:
import torch
# this looks neat: https://github.com/patrick-kidger/torchtyping

def command2vec(command: Tuple[str, int]) -> torch.Tensor:
  """
  Converts a navigation command into a vector that can be added to our current position
  to get the value of our position after following the direction
  """
  # each direction represents a vector dimension (0 for vertical, 1 for horizontal)
  # and a polarity (1 for forward/up, -1 for down. nb: reverse is not a supported command)
  v_dim = {'up':0, 'down':0, 'forward':1}
  v_polarity = {'up':-1, 'down':1, 'forward':1}

  direction, value = command
  d = v_dim[direction]
  p = v_polarity[direction]

  v = torch.zeros(2).unsqueeze(0)
  v[:,d] = p*value
  return v

def reduce_commands(commands_parsed: CommandsList) -> torch.Tensor:
  return torch.cat([command2vec(c) for c in commands_parsed], dim=0).sum(dim=0)

assert reduce_commands(test_parsed).prod() == 150

In [26]:
# part 1 solution

# 'data' variable defined at bottom
parsed = parse_input(data)
reduce_commands(parsed).prod().item()

1698735.0

## Part Two

Based on your calculations, the planned course doesn't seem to make any sense. You find the submarine manual and discover that the process is actually slightly more complicated.

In addition to horizontal position and depth, you'll also need to track a third value, aim, which also starts at 0. The commands also mean something entirely different than you first thought:

- `down X` increases your aim by X units.
- `up X` decreases your aim by X units.
- `forward X` does two things:
  - It increases your horizontal position by X units.
  - It increases your depth by your aim multiplied by X.

Again note that since you're on a submarine, down and up do the opposite of what you might expect: "down" means aiming in the positive direction.

Now, the above example does something different:

- `forward 5` adds 5 to your horizontal position, a total of 5. Because your aim is 0, your depth does not change.
- `down 5` adds 5 to your aim, resulting in a value of 5.
- `forward 8` adds 8 to your horizontal position, a total of 13. Because your aim is 5, your depth increases by 8*5=40.
- `up 3` decreases your aim by 3, resulting in a value of 2.
down 8 adds 8 to your aim, resulting in a value of 10.
- `forward 2` adds 2 to your horizontal position, a total of 15. Because your aim is 10, your depth increases by 2*10=20 to a total of 60.

After following these new instructions, you would have a horizontal position of 15 and a depth of 60. (Multiplying these produces 900.)

Using this new interpretation of the commands, calculate the horizontal position and depth you would have after following the planned course. What do you get if you multiply your final horizontal position by your final depth?

In [31]:
# I don't think we can do this vectorized like before. 'Aim' is a state attribute we need to track. No problem.

class Navigator:
  def __init__(self):
    self.x_pos = 0
    self.y_pos = 0
    self.aim = 0 # I think this is really just 'slope'. whatever.

  def process_command(self, command: Command):
    direction, value = command
    polarity = 1
    if direction == 'up':
      polarity = -1
    
    if direction in ('up', 'down'):
      self.aim += value*polarity
    elif direction == 'forward':
      self.x_pos += value
      self.y_pos += value*self.aim 
    else:
      raise NotImplementedError
    
nav = Navigator()
for c in test_parsed:
  nav.process_command(c)

assert (nav.x_pos, nav.y_pos) == (15, 60)

In [37]:
# part 2 solution

nav = Navigator()
for c in parsed:
  nav.process_command(c)

nav.x_pos * nav.y_pos

1594785890

In [24]:
data = """
forward 7
down 8
forward 5
down 3
forward 6
down 9
down 2
forward 1
down 2
down 9
forward 8
down 8
up 6
forward 8
down 3
forward 9
up 3
up 7
forward 6
up 3
down 5
up 9
down 6
up 1
forward 7
up 2
forward 3
forward 3
up 9
down 9
up 4
forward 9
down 8
up 1
forward 8
forward 6
up 8
down 5
up 7
up 5
up 5
up 4
up 3
forward 8
forward 9
down 9
forward 7
forward 1
forward 2
forward 4
up 2
forward 4
up 3
down 7
forward 6
down 9
down 3
down 4
forward 4
forward 8
forward 9
down 8
forward 3
forward 3
forward 7
forward 3
down 7
forward 6
down 8
forward 6
up 3
forward 9
down 7
down 6
down 6
down 5
forward 7
down 9
down 9
forward 6
forward 5
forward 6
up 3
forward 1
forward 8
down 5
up 2
up 7
up 9
forward 5
forward 3
down 6
forward 8
forward 6
down 8
down 2
forward 5
down 8
forward 7
down 6
down 2
forward 3
down 6
up 3
down 4
down 5
down 6
forward 9
down 8
down 9
forward 3
up 5
up 2
up 6
forward 3
down 4
down 6
forward 7
forward 9
up 3
up 8
up 4
down 4
up 9
up 9
down 1
forward 6
down 8
forward 4
down 1
forward 8
up 1
up 4
forward 8
forward 8
down 9
forward 6
forward 4
forward 6
forward 7
down 2
down 6
forward 1
forward 8
up 4
forward 3
forward 5
down 2
forward 5
forward 6
forward 9
forward 3
down 6
down 5
down 1
forward 2
forward 7
forward 2
forward 2
down 3
down 5
forward 2
forward 7
forward 6
forward 8
up 8
forward 9
down 4
down 7
forward 4
down 9
up 1
forward 5
forward 5
up 3
forward 3
up 2
forward 4
down 6
down 6
down 5
forward 5
forward 9
down 8
forward 9
forward 5
forward 4
up 4
forward 9
forward 2
forward 2
up 8
up 4
forward 8
forward 3
forward 6
forward 1
up 8
forward 2
forward 6
up 6
down 5
up 9
forward 1
down 5
forward 5
down 1
down 6
up 3
up 4
down 3
forward 3
down 2
forward 3
down 1
down 6
forward 6
down 7
up 2
forward 2
forward 9
forward 7
up 5
up 8
up 8
down 4
up 5
forward 6
up 1
forward 4
forward 5
up 4
up 8
up 2
down 4
up 3
down 2
forward 1
down 8
forward 5
up 2
forward 7
down 4
down 5
forward 4
forward 7
down 9
up 9
forward 2
up 3
up 8
up 3
up 3
down 3
forward 1
forward 2
forward 6
forward 3
up 4
forward 1
down 2
forward 4
down 9
forward 5
down 6
down 5
forward 9
forward 3
forward 9
forward 3
forward 8
down 3
forward 5
down 1
forward 7
forward 6
down 7
down 8
up 9
down 2
forward 7
down 9
forward 5
forward 3
down 1
forward 6
down 6
up 8
down 6
up 9
forward 2
down 1
forward 2
down 8
down 2
down 6
up 9
down 3
down 5
up 4
down 4
down 3
up 2
forward 2
down 1
down 2
forward 8
forward 3
down 6
forward 9
up 5
down 5
down 1
forward 1
down 3
down 7
down 5
forward 6
up 4
down 2
forward 8
down 9
up 8
forward 9
forward 5
down 6
up 4
forward 1
forward 2
forward 5
up 3
up 2
up 8
up 6
forward 9
down 3
down 7
down 7
up 5
forward 1
down 3
forward 9
down 5
up 6
down 4
up 5
up 4
up 9
forward 2
up 7
down 7
up 3
up 1
forward 6
down 1
down 9
down 9
down 4
forward 1
forward 6
up 9
down 3
up 8
forward 7
forward 3
down 6
up 9
forward 3
up 9
down 3
up 6
down 4
up 3
down 2
forward 9
forward 2
down 1
forward 4
down 5
up 4
forward 6
down 4
down 8
forward 4
down 8
down 9
up 4
up 7
down 7
forward 6
up 9
down 9
down 7
forward 3
up 4
up 8
down 2
down 5
down 9
forward 3
forward 2
up 6
down 8
forward 2
forward 7
up 4
forward 4
forward 7
forward 7
forward 3
down 2
down 6
up 6
down 8
down 4
forward 2
down 2
forward 2
forward 7
forward 2
down 1
down 1
down 3
down 3
forward 4
down 9
forward 2
forward 4
forward 1
forward 7
forward 3
down 1
down 7
forward 9
forward 7
up 2
down 5
forward 4
down 8
down 8
forward 5
down 3
up 6
down 3
forward 3
forward 8
up 9
up 8
up 7
forward 4
forward 6
forward 6
down 6
forward 1
up 8
down 6
forward 1
down 3
down 2
up 7
forward 5
down 2
down 6
down 8
up 2
forward 6
forward 3
forward 9
up 8
up 2
down 4
forward 3
down 9
forward 9
forward 2
down 7
forward 6
up 4
forward 1
down 5
forward 7
up 6
down 2
up 2
forward 6
down 6
forward 1
forward 5
down 8
down 2
up 8
forward 2
forward 2
down 4
down 9
forward 9
down 9
forward 5
down 6
down 8
forward 4
down 8
up 8
forward 5
forward 8
up 1
up 3
forward 3
down 7
down 8
up 3
forward 1
up 2
up 9
down 8
up 5
forward 7
forward 7
down 7
down 3
forward 1
forward 3
forward 7
down 8
forward 7
forward 6
down 5
down 8
down 1
forward 4
down 5
down 9
forward 1
forward 3
forward 3
up 2
forward 5
down 3
forward 2
down 7
up 7
up 7
forward 5
up 1
down 8
up 4
up 6
forward 4
forward 4
forward 4
down 7
down 9
down 7
forward 8
down 6
down 3
forward 5
down 6
forward 5
forward 3
forward 9
forward 9
up 6
down 9
up 4
up 1
down 5
down 2
up 9
forward 9
forward 8
up 5
down 6
up 6
down 4
down 6
up 3
forward 2
up 7
up 1
up 4
down 7
up 2
forward 7
up 9
up 3
forward 9
forward 6
forward 1
up 5
down 4
down 3
up 3
forward 5
down 3
down 4
up 4
up 9
up 6
forward 5
down 9
down 7
forward 4
forward 9
down 9
down 9
down 8
forward 1
up 6
down 7
down 1
down 3
down 3
forward 1
forward 8
up 1
forward 2
down 2
down 5
down 9
forward 1
forward 4
down 7
forward 1
down 1
down 8
down 3
forward 3
down 9
up 9
down 7
down 1
down 2
down 2
forward 1
up 3
up 1
down 5
forward 5
down 4
down 4
down 3
up 9
forward 9
down 3
forward 6
forward 1
forward 2
forward 8
forward 9
down 3
down 5
up 7
down 4
forward 8
up 9
forward 9
forward 2
down 2
forward 3
down 4
forward 2
forward 6
forward 9
up 8
down 1
down 4
up 9
forward 2
down 9
forward 4
down 7
up 5
down 2
forward 5
forward 7
forward 9
forward 1
down 1
forward 1
down 3
forward 5
up 8
down 8
forward 1
down 5
up 3
down 5
forward 3
forward 7
forward 2
down 2
forward 7
forward 4
forward 4
down 3
forward 5
down 5
down 9
up 4
down 6
down 5
forward 5
forward 9
down 9
forward 9
down 6
forward 4
down 5
down 8
up 9
down 7
up 7
forward 4
down 6
forward 5
forward 4
up 7
forward 9
forward 7
up 7
down 8
down 1
forward 3
down 3
down 6
up 6
up 5
up 8
down 7
forward 6
down 9
forward 9
forward 6
up 7
down 4
down 5
down 8
forward 9
forward 8
forward 2
forward 4
up 6
down 7
forward 8
down 9
forward 3
forward 1
forward 4
forward 8
down 4
down 1
forward 5
down 7
down 3
up 2
up 5
down 6
forward 5
down 9
down 3
forward 8
up 5
down 6
forward 3
down 5
forward 1
down 3
up 3
forward 9
forward 9
forward 8
down 4
down 1
forward 7
forward 7
down 9
up 2
forward 2
forward 2
forward 1
forward 2
down 2
down 7
forward 5
forward 3
forward 2
down 1
down 6
up 4
down 3
forward 2
down 2
down 2
forward 9
up 8
forward 9
down 8
up 8
down 8
down 1
forward 9
up 7
up 6
down 3
forward 5
up 9
forward 5
forward 8
up 3
forward 8
forward 9
forward 2
forward 4
down 7
forward 1
forward 1
down 7
forward 8
forward 4
forward 2
forward 7
forward 6
forward 5
down 8
down 9
up 9
forward 1
forward 4
down 1
forward 6
up 4
forward 8
forward 6
up 5
forward 9
down 8
down 4
down 9
forward 2
up 4
down 2
forward 4
down 9
forward 3
down 8
forward 7
forward 3
down 7
down 7
down 8
forward 8
down 2
forward 7
up 7
down 5
down 7
down 5
up 9
up 8
down 1
down 9
forward 7
up 2
forward 1
forward 3
forward 8
down 5
forward 7
up 6
forward 5
up 5
forward 4
down 6
down 4
forward 1
up 1
forward 2
forward 2
forward 8
forward 7
down 4
down 1
forward 5
forward 9
up 4
down 4
up 9
up 4
down 8
down 7
forward 1
down 8
forward 9
down 4
forward 5
forward 2
down 1
forward 7
forward 4
down 9
forward 8
down 1
forward 7
forward 6
up 9
forward 6
forward 6
forward 8
forward 3
up 1
forward 9
down 3
down 9
down 8
forward 5
down 7
forward 6
forward 1
up 1
up 3
up 7
forward 1
forward 8
forward 6
forward 9
down 6
forward 9
down 2
up 2
down 7
down 7
forward 8
down 2
down 3
forward 7
forward 5
down 3
down 1
up 8
up 6
down 3
forward 1
forward 6
forward 4
forward 7
down 3
down 9
up 6
down 7
forward 9
forward 2
forward 7
down 2
down 8
down 1
down 9
up 8
forward 3
down 9
forward 6
down 4
forward 1
up 8
up 4
down 5
forward 2
forward 9
forward 5
forward 6
down 4
up 4
down 6
down 8
down 2
down 7
down 2
up 8
forward 4
down 7
forward 2
down 5
forward 3
forward 1
down 2
up 3
down 2
forward 1
forward 2
down 6
down 2
down 9
forward 9
down 1
up 2
down 4
forward 1
forward 6
forward 8
forward 3
""".strip()